# Esercizio 8

Prendere in input il file `embl` `M10051.txt`, contenente un mRNA (cioé il trascritto espresso da un gene) e determinare tutte le proteine più lunghe di 1000 amminoacidi che il trascritto può esprimere. Per ognuna di esse determinare lo start e l'end del relativo Open Reading Frame.

Un Open Reading Frame (ORF) una sottostringa dell'mRNA, con lunghezza multiplo di 3, che inizia con lo start codon `atg`, finisce con uno stop codon (`taa`, `tag`, `tga`) e non contiene al suo interno uno stop codon, cioè non contiene una sottostringa, uguale a `taa`, `tag` oppure `tga`, che sia in *frame* con lo start codon `atg`.

---

In [ ]:
import Bio

## Leggere la sequenza del trascritto

In [ ]:
from Bio import SeqIO

In [ ]:
transcript_sequence = SeqIO.read('./M10051.txt', 'embl').seq

## Eseguire la traduzione di ognuno dei tre frame di lettura del trascritto

- Primo frame di lettura --> tutto il trascritto
- Secondo frame di lettura --> tutto il trascritto tranne la prima base
- Terzo frame di lettura --> tutto il trascritto tranne le prime due basi

Esempio di trascritto:

        cgtgaggacggatgcccttgatgtcatgacggtgaggtaactctgagt

Frame di lettura:

        cgtgaggacggatgcccttgatgtcatgacggtgaggtaactctgagt  I
         gtgaggacggatgcccttgatgtcatgacggtgaggtaactctgagt  II
          tgaggacggatgcccttgatgtcatgacggtgaggtaactctgagt  III


In [ ]:
frame_translations = [transcript_sequence[i:].translate() for i in {0,1,2}]

In [ ]:
frame_translations

## Separare ognuna delle traduzioni per stop codon

        REDGCP*CHDGEVTLS	
        VRTDALDVMTVR*L*		
        *GRMPLMS*R*GNSE		
        
Separazione per carattere `*`

        REDGCP   CHDGEVTLS	
        VRTDALDVMTVR   L		
        GRMPLMS  R  GNSE	


In [ ]:
chunk_list = [frame.split('*') for frame in frame_translations]

In [ ]:
chunk_list

## Estrarre da ogni stringa i suffissi lunghi almeno 1000 caratteri che iniziano con `M` 


        REDGCP   CHDGEVTLS	
        VRTDALDVMTVR   L		
        GRMPLMS  R  GNSE	
        
Suffissi che iniziano con `M`
        
        MTVR   MPLMS   MS

Importiamo il modulo re per usare le espressioni regolari.

In [ ]:
import re

Estraiamo tutti i suffissi (lunghi almeno 1000 caratteri) che iniziano con `M` dai chunk dei tre frame di lettura

In [ ]:
candidate_proteins = []

for (f, chunk_list_for_a_frame) in enumerate(chunk_list):
    offset = 0
    for chunk in chunk_list_for_a_frame:
        #print(chunk)
        occ_list = list(re.finditer('M', str(chunk)))
        for occ in occ_list:
            protein = chunk[occ.start():]
            if len(protein) >= 1000:
                candidate_proteins.append([protein, f, offset, occ.start()])
                
        offset = offset + len(chunk) + 1

In [ ]:
candidate_proteins

### Per ognuna delle proteine candidate risaliamo al relativo Open Reading Frame (ORF) sul trascritto

In [ ]:
for cp in candidate_proteins:
    protein = cp[0]
    f = cp[1]
    offset = cp[2]
    p_start = cp[3]
    
    orf_start = f + (offset + p_start)*3 + 1
    orf_end = orf_start + len(protein)*3 + 2
    print(protein)
    print(orf_start, orf_end)
    print('************************')

### Proteina espressa dal trascritto secondo l'annotazione